<div style="display:flex; align-items:flex-start; margin-bottom:1rem;">
  <!-- Left: Book cover -->
  <img
    src="https://i.imgur.com/ITL8dZE.jpeg"
    style="width:35%; margin-right:1rem; border-radius:4px; box-shadow:0 2px 6px rgba(0,0,0,0.1);"
    alt="Book Cover"/>
  <!-- Right: Metadata -->
  <div style="flex:1;">
    <!-- O'Reilly logo above title -->
    <div style="display:flex; flex-direction:column; align-items:flex-start; margin-bottom:0.75rem;">
      <img
        src="https://cdn.oreillystatic.com/images/sitewide-headers/oreilly_logo_mark_red.svg"
        style="height:2rem; margin-bottom:0.25rem;"
        alt="O‘Reilly"/>
      <span style="font-size:1.75rem; font-weight:bold; line-height:1.2;">
        AI, ML and GenAI in the Lakehouse
      </span>
    </div>
    <!-- Details, now each on its own line -->
    <div style="font-size:0.9rem; color:#555; margin-bottom:1rem; line-height:1.4;">
      <div><strong>Name:</strong>10-01-Financial Agents</div>
      <div><strong>Author:</strong> Bennie Haelen</div>
      <div><strong>Date:</strong> 7-26-2025</div>
    </div>
    <!-- Purpose -->
    <div style="font-weight:600; margin-bottom:0.75rem;">
      Purpose: This notebook contains the code for the React Notebook
    </div>
    <!-- Outline -->
    <div style="margin-top:0;">
      <h3 style="margin:0 0 0.25rem;">Table of Contents</h3>
      <ol style="padding-left:1.25rem; margin:0; color:#333;">
        <li>Fetch Wikipedia articles and load them into a DataFrame</li>
        <li>Extract/clean the text content-split it into manageable chunks</li>
        <li>Calculate the embeddings</li>
        <li>Store the embeddings in a Delta file</li>
      </ol>
    </div>
  </div>
</div>


#Notebook Initialization

##Install the required libraries

In [0]:
# For full agent mode  
!pip install --upgrade langchain langchain-openai pandas 
 
dbutils.library.restartPython()

##Import Statements

In [0]:
"""
React Agent for Databricks Lakehouse Intelligence
Author: Bennie Haelen
Book: AI and ML in a Lakehouse

This Notebook provides a complete, example of a React agent designed 
to interact with a Databricks Lakehouse. It uses the Databricks
Connect library to query data, includes multiple tools for analysis, and
features both a live and a demo mode for educational purposes.
"""

import os
import re
import pandas as pd
from typing import Dict, Any, Optional

# LangChain components for building the agent
from langchain_openai import ChatOpenAI
from langchain_core.tools import BaseTool
from langchain_core.prompts import PromptTemplate
from langchain_core.callbacks import CallbackManagerForToolRun
from langchain.agents import create_react_agent, AgentExecutor

# Databricks Connect for querying the Lakehouse
from databricks.connect import DatabricksSession
from databricks.sdk.core import Config

##Perform MLflow Autologging

In [0]:
import mlflow
mlflow.langchain.autolog()

#Setup the Lakehouse Environment

In [0]:
# --- Utility Function to Set Up the Environment ---

def setup_lakehouse_environment():
    """
    Creates the necessary schemas and tables in the Databricks environment
    and populates them with sample data. This function makes the example
    self-contained and runnable.
    """
    print("Setting up the Databricks Lakehouse environment...")
    # spark = DatabricksSession.builder.getOrCreate()

    # Dynamically get the current catalog to ensure portability
    current_catalog = spark.catalog.currentCatalog()
    print(f"Using current catalog: '{current_catalog}'")

    # Define schemas within the current catalog
    schemas = [f"{current_catalog}.bronze", f"{current_catalog}.silver", f"{current_catalog}.gold"]
    for schema_name in schemas:
        spark.sql(f"CREATE SCHEMA IF NOT EXISTS {schema_name}")
    print("Schemas created successfully.")

    # --- Create and Populate Tables ---

    # GOLD LAYER
    gold_data = {
        'month': pd.to_datetime(['2025-07-31', '2025-06-30', '2025-05-31']),
        'avg_engagement_premium': [71.0, 83.0, 85.0],
        'avg_engagement_standard': [60.0, 62.0, 64.0],
        'total_revenue': [250000.0, 245000.0, 240000.0],
        'premium_customer_count': [1050, 1045, 1040]
    }
    gold_df = pd.DataFrame(gold_data)
    spark.createDataFrame(gold_df).write.mode("overwrite").saveAsTable(f"{current_catalog}.gold.monthly_engagement")

    # SILVER LAYER
    silver_customers_data = {
        'customer_id': range(101, 111),
        'tier': ['premium', 'standard'] * 5,
        'signup_date': pd.to_datetime(pd.date_range('2024-01-01', periods=10)),
        'lifetime_value': [500 + i*100 for i in range(10)]
    }
    silver_customers_df = pd.DataFrame(silver_customers_data)
    spark.createDataFrame(silver_customers_df).write.mode("overwrite").saveAsTable(f"{current_catalog}.silver.customer_profiles")

    silver_sessions_data = {
        'session_id': [f's{i}' for i in range(20)],
        'customer_id': [101 + (i % 10) for i in range(20)],
        'session_date': pd.to_datetime(pd.date_range('2025-07-15', periods=20)),
        'duration_minutes': [10 + (i % 15) for i in range(20)],
        'converted': [i % 3 == 0 for i in range(20)]
    }
    silver_sessions_df = pd.DataFrame(silver_sessions_data)
    spark.createDataFrame(silver_sessions_df).write.mode("overwrite").saveAsTable(f"{current_catalog}.silver.session_summaries")

    # BRONZE LAYER
    bronze_data = {
        'user_id': [101 + (i % 10) for i in range(50)],
        'event_type': ['page_view', 'click', 'page_view', 'add_to_cart', 'purchase'] * 10,
        'timestamp': pd.to_datetime(pd.date_range('2025-07-31 10:00', periods=50, freq='T'))
    }
    bronze_df = pd.DataFrame(bronze_data)
    spark.createDataFrame(bronze_df).write.mode("overwrite").saveAsTable(f"{current_catalog}.bronze.clickstream_raw")
    
    print("All tables created and populated successfully.")


#Setup Tools

##Databricks Query Tool

In [0]:
class DatabricksQueryTool(BaseTool):
    """
    Tool for executing SQL queries against the Databricks Lakehouse.
    """
    name: str = "databricks_sql_query"
    description: str = """
    Executes a SQL query against the Databricks Lakehouse.
    Input must be a valid Databricks SQL query.
    Example: "SELECT * FROM my_catalog.silver.customer_profiles WHERE tier = 'premium' LIMIT 10"
    """

    # Corrected code for DatabricksQueryTool
    def _run(
        self,
        query: str,
        run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Execute a SQL query and return the result as a JSON string."""
        print(f"\nExecuting Databricks Query with original input:\n---\n{query}\n---\n")
        try:
            # More robust cleaning to handle markdown code blocks from the LLM
            match = re.search(r"```(?:sql)?\s*(.*?)\s*```", query, re.DOTALL)
            if match:
                clean_query = match.group(1).strip()
            else:
                # THIS IS THE FIX: Only strip whitespace, not quotes.
                clean_query = query.strip()
            
            print(f"Cleaned Query to be executed: {clean_query}")

            spark = DatabricksSession.builder.getOrCreate()
            result_df = spark.sql(clean_query).toPandas()

            if result_df.empty:
                return "Query returned no results."
            
            return result_df.to_json(orient='records')

        except Exception as e:
            return f"Error executing Databricks query: {str(e)}. Please check your SQL syntax and table/column names."

##Schema Tool

In [0]:
class SchemaInfoTool(BaseTool):
    """Tool to get Lakehouse schema information."""
    name: str = "get_lakehouse_schema"
    description: str = "Provides the schema for the key tables in the bronze, silver, and gold layers of the Lakehouse."

    def _run(
        self,
        query: str, # The agent might pass an argument, but we ignore it
        run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Return a pre-defined schema string for the agent to use."""
        spark = DatabricksSession.builder.getOrCreate()
        current_catalog = spark.catalog.currentCatalog()
        
        return f"""
        DATABRICKS LAKEHOUSE SCHEMA (CURRENT CATALOG: {current_catalog}):

        GOLD LAYER (Business-Level Aggregates):
        - {current_catalog}.gold.monthly_engagement:
            - month (date): The month of the aggregated data.
            - avg_engagement_premium (float): Average engagement score for premium customers.
            - avg_engagement_standard (float): Average engagement score for standard customers.
            - total_revenue (float): Total revenue for the month.
            - premium_customer_count (int): Number of premium customers.

        SILVER LAYER (Cleaned & Enriched Data):
        - {current_catalog}.silver.customer_profiles:
            - customer_id (int): Unique identifier for a customer.
            - tier (string): Customer tier ('premium' or 'standard').
            - signup_date (date): The date the customer signed up.
            - lifetime_value (float): The total amount spent by the customer.
        - {current_catalog}.silver.session_summaries:
            - session_id (string): Unique identifier for a session.
            - customer_id (int): The customer who initiated the session.
            - session_date (date): The date of the session.
            - duration_minutes (int): The duration of the session in minutes.
            - converted (boolean): Whether the session resulted in a purchase.

        BRONZE LAYER (Raw Data):
        - {current_catalog}.bronze.clickstream_raw:
            - user_id (int): The ID of the user.
            - event_type (string): The type of event (e.g., 'page_view', 'click').
            - timestamp (timestamp): The exact time of the event.
        """


##Trend Calculator Tool

In [0]:
import json # Make sure to import the json library at the top of your notebook

# --- Corrected Trend Calculator Tool ---

class TrendCalculatorTool(BaseTool):
    """Tool to calculate trends and percentage changes."""
    name: str = "calculate_trend"
    description: str = """
    Calculates the percentage change and trend direction between two numeric values.
    Input must be a comma-separated string in the format: 'metric_name,current_value,previous_value'
    Example: 'Engagement Score,71,83'
    """

    def _run(
        self,
        query: str,
        run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Calculate the trend and return a descriptive JSON string."""
        try:
            clean_query = query.strip().strip("'\"")
            
            parts = clean_query.split(',')
            if len(parts) != 3:
                return "Error: Input format must be 'metric_name,current_value,previous_value'"

            metric_name, current_str, previous_str = parts

            current_value = float(current_str.strip().strip("'\""))
            previous_value = float(previous_str.strip().strip("'\""))

            if previous_value == 0:
                change = float('inf') if current_value > 0 else 0.0
            else:
                change = ((current_value - previous_value) / previous_value) * 100
            
            direction = "increased" if change > 0 else "decreased"

            # THE FIX: Return a JSON string instead of a sentence.
            # This forces the agent to process the data in its next thought step.
            result = {
                "metric_name": metric_name.strip(),
                "trend": direction,
                "percentage_change": round(abs(change), 1),
                "from_value": previous_value,
                "to_value": current_value
            }
            return json.dumps(result)

        except ValueError as e:
            return f"Error: Invalid number provided. Could not convert to float. Details: {e}"
        except Exception as e:
            return f"An unexpected error occurred during trend calculation: {str(e)}"

#React Agent Implementation

In [0]:
# --- React Agent Implementation ---

class LakehouseReactAgent:
    """A React Agent for intelligent exploration of a Databricks Lakehouse."""

    def __init__(self, openai_api_key: str):
        """Initializes the agent, tools, and LLM."""
        self.llm = ChatOpenAI(
            temperature=0,
            model_name="gpt-4o",
            openai_api_key=openai_api_key
        )

        self.tools = [
            DatabricksQueryTool(),
            SchemaInfoTool(),
            TrendCalculatorTool()
        ]

        # --- NEW: Manually provide the tool names to the prompt ---

        # 1. Define the original prompt string with the {tool_names} placeholder.
        prompt_with_tool_names = PromptTemplate.from_template("""
You are an expert data analyst specializing in the Databricks Lakehouse.
Your primary goal is to help users understand their business data by forming hypotheses,
querying the Lakehouse, and providing clear, actionable insights.

**Your Thought Process:**
1.  **Understand the Goal:** Fully understand the user's question.
2.  **Consult the Schema:** Always start by using the `get_lakehouse_schema` tool to see the available tables and columns. This is mandatory.
3.  **Formulate a Plan (Initial):** Start with high-level data (Gold layer) to confirm the primary metric in the user's question.
4.  **Execute and Observe:** Use the `databricks_sql_query` tool to execute one query at a time.
5.  **Drill-Down and Hypothesize:**
    * After confirming the initial finding, **do not stop**. Your main goal is to find the "why".
    * Formulate a hypothesis about the cause. For example: "Perhaps the engagement drop is due to lower session durations."
    * To test your hypothesis, plan a query against a more granular table (e.g., the Silver layer).
6.  **Synthesize and Answer:** Once you have gathered enough information from multiple layers, provide a final, comprehensive answer.

**Final Answer Structure:**
Your final answer MUST be structured with the following markdown sections:
- `**Key Finding:**` A one-sentence summary of the main metric change.
- `**Business Impact:**` A short paragraph explaining why this change is important.
- `**Recommended Next Steps & Hypotheses:**` A numbered list of concrete actions or queries to investigate further. This is the most important part of your answer.

**Critical Rules:**
- **SQL Syntax:** Use Databricks SQL. Use single quotes (') for strings. Use the full three-level namespace.
- **THE MOST RELIABLE WAY TO FILTER DATES:** To compare a date/timestamp column with a string, always CAST the string to a DATE. Example: `WHERE month = CAST('2025-07-31' AS DATE)`.
- **Tool Input:** Ensure your 'Action Input' is a clean, raw string for the tool.

**Tools Available:**
{tools}

**Interaction Format:**

Question: The user's question you need to answer.
Thought: Your reasoning and plan for the next action.
Action: The tool to use, chosen from [{tool_names}].
Action Input: The exact input for the chosen tool.
Observation: The result returned by the tool.
... (this Thought/Action/Action Input/Observation cycle can repeat multiple times)
Thought: I have now gathered all the necessary information to answer the user's question.
Final Answer: Your final, well-structured answer to the original question.

---

Question: {input}
Thought: {agent_scratchpad}
""")

        # 2. Get the tool names from your tools list.
        tool_names = ", ".join([tool.name for tool in self.tools])

        # 3. Use the .partial() method to "pre-fill" the tool_names variable.
        self.prompt = prompt_with_tool_names.partial(tool_names=tool_names)
        self.agent = create_react_agent(
            llm=self.llm,
            tools=self.tools,
            prompt=self.prompt
        )

        self.agent_executor = AgentExecutor(
            agent=self.agent,
            tools=self.tools,
            verbose=True,
            max_iterations=12,
            handle_parsing_errors="Check your input format and try again.",
            return_intermediate_steps=True
        )

    def analyze(self, question: str) -> Dict[str, Any]:
        """Analyzes a business question using the React Agent."""
        try:
            result = self.agent_executor.invoke({"input": question})
            return {
                "answer": result.get("output", "No output generated."),
                "intermediate_steps": result.get("intermediate_steps", []),
                "success": True
            }
        except Exception as e:
            return {
                "answer": f"An error occurred during agent execution: {str(e)}",
                "intermediate_steps": [],
                "success": False
            }

##Demo Agent (Delete Later)

In [0]:
# --- Demo Agent for Educational Purposes ---

class DemoReactAgent:
    """
    A simulated React Agent that demonstrates the reasoning process
    without requiring API keys or a live Databricks connection.
    Perfect for book examples and offline demonstrations.
    """
    def __init__(self):
        self.tools = {
            'get_lakehouse_schema': SchemaInfoTool(),
            'calculate_trend': TrendCalculatorTool()
        }

    def analyze(self, question: str) -> Dict[str, Any]:
        """Simulates the step-by-step reasoning of the React Agent."""
        print(f"Analyzing Question: {question}\n")
        print("--- DEMO AGENT REASONING LOG ---")

        steps = []
        
        thought1 = "First, I need to understand the available data. I will use the get_lakehouse_schema tool."
        print(f"Thought: {thought1}")
        action1 = "get_lakehouse_schema"
        print(f"Action: {action1}")
        action_input1 = ""
        print(f"Action Input: {action_input1}")
        observation1 = self.tools['get_lakehouse_schema']._run(action_input1)
        print(f"Observation: {observation1}")
        steps.append((thought1, action1, action_input1, observation1))

        thought2 = "The user is asking about premium customer engagement. The `gold.monthly_engagement` table looks like the best place to start. I'll get the last two months of data to see the trend."
        print(f"Thought: {thought2}")
        action2 = "databricks_sql_query"
        print(f"Action: {action2}")
        action_input2 = "SELECT month, avg_engagement_premium FROM my_catalog.gold.monthly_engagement ORDER BY month DESC LIMIT 2"
        print(f"Action Input: {action_input2}")
        observation2 = '[{"month": "2025-07-31", "avg_engagement_premium": 71.0}, {"month": "2025-06-30", "avg_engagement_premium": 83.0}]'
        print(f"Observation: {observation2}")
        steps.append((thought2, action2, action_input2, observation2))

        thought3 = "The observation shows a drop in engagement from 83.0 to 71.0. I need to calculate the exact percentage change to quantify this drop."
        print(f"Thought: {thought3}")
        action3 = "calculate_trend"
        print(f"Action: {action3}")
        action_input3 = "Premium Engagement,71.0,83.0"
        print(f"Action Input: {action_input3}")
        observation3 = self.tools['calculate_trend']._run(action_input3)
        print(f"Observation: {observation3}")
        steps.append((thought3, action3, action_input3, observation3))
        
        thought4 = "I have confirmed the drop and quantified it. I now have enough information to provide a final answer."
        print(f"Thought: {thought4}")
        final_answer = """
**Analysis of Premium Customer Engagement Drop**

**Key Finding:**
There was a significant **14.5% decrease** in the average engagement score for premium customers, falling from 83.0 in June 2025 to 71.0 in July 2025.

**Business Impact:**
A drop of this magnitude is a strong indicator of declining customer satisfaction or perceived value among your most important customer segment. This could be a leading indicator of future churn and revenue loss if not addressed promptly.

**Recommended Next Steps & Hypotheses:**
1.  **Drill-Down Analysis:** Investigate if this drop is correlated with a decrease in session duration or conversion rates by querying the `silver.session_summaries` table.
2.  **Product Changes:** Was a key feature deprecated or changed in late June or early July?
3.  **Marketing Campaigns:** Did a competitor launch a new, attractive offer that could be pulling customers away?
        """
        print(f"Final Answer: {final_answer}")

        return {
            "answer": final_answer,
            "intermediate_steps": steps,
            "success": True
        }


#ReAct Agent Implementation

In [0]:


# --- Main Execution Block ---

"""Main function to run the agent."""
print("="*80)
print("🤖 Welcome to the React Agent for Databricks Lakehouse Intelligence 🤖")
print("="*80)
print("This script demonstrates how an AI agent can intelligently query and analyze data.")
print("\nChoose an execution mode:")
print("1. Demo Mode (Recommended for first-time use, no API key or Databricks setup needed)")
print("2. Live Agent Mode (Requires OpenAI API key and Databricks Connect configuration)")

print("\n--- Live Agent Mode ---")
try:
    openai_api_key = dbutils.secrets.get(scope="book", key="OPENAI_API_KEY")
    print(f"OpenAI API key retrieved successfully.")                                
except Exception as e:
    print("❌ ERROR: Could not retrieve OpenAI API key from Databricks Secrets.")
    print("Please ensure the secret scope 'my-react-agent' and key 'openai-api-key' are configured correctly.")
    exit(0)

if not openai_api_key:
    print("❌ ERROR: OPENAI_API_KEY environment variable not set.")
    print("Please set it to run the live agent. Exiting.")
    exit(0)

try:
    print("Verifying Databricks Connect configuration...")
    spark = DatabricksSession.builder.getOrCreate()
    print("✅ Databricks session created successfully.")
    
    # Set up the mock data environment for the live session
    setup_lakehouse_environment()

except Exception as e:
    print(f"❌ ERROR: Databricks Connect is not configured correctly. {e}")
    print("Please run 'databricks configure' in your terminal. Exiting.")
    exit(0)

agent = LakehouseReactAgent(openai_api_key=openai_api_key)
question = "Why did our premium customers' engagement drop last month?"
print(f"\n🚀 Asking the live agent: '{question}'")
result = agent.analyze(question)
print("\n--- FINAL AGENT RESPONSE ---")
print(result["answer"])


